In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import json
from collections import defaultdict
import seaborn as sns


try:
    import nibabel as nib
    NIBABEL_AVAILABLE = True
    print(" nibabel loaded successfully")
except ImportError:
    NIBABEL_AVAILABLE = False
    print(" nibabel not available - some features will be limited")


try:
    from scipy import ndimage
    SCIPY_AVAILABLE = True
except ImportError:
    SCIPY_AVAILABLE = False

print(" Libraries loaded successfully")
print(f" Working directory: {os.getcwd()}")

In [ ]:
import os
import nibabel as nib
import numpy as np


data_root = r"C:\Users\KamrunNaher\Downloads\BRaTS2021_Training_Data"
explorer = SimpleBRaTSExplorer(data_root)
explorer.discover_patients()

print("\n Checking all T1CE files for issues...\n")

problematic_patients = []

for i, patient_id in enumerate(explorer.patient_dirs):
    patient_path = os.path.join(data_root, patient_id)
    t1ce_file = os.path.join(patient_path, f"{patient_id}_t1ce.nii.gz")

    if not os.path.exists(t1ce_file):
        print(f" {patient_id}: T1CE file missing!")
        problematic_patients.append(patient_id)
        continue

    try:
        img = nib.load(t1ce_file)
        data = img.get_fdata()

        if np.count_nonzero(data) == 0:
            print(f" {patient_id}: T1CE file exists but is empty (all zeros)")
            problematic_patients.append(patient_id)
        elif np.isnan(data).any():
            print(f" {patient_id}: T1CE contains NaNs")
            problematic_patients.append(patient_id)
        else:

            print(f" {patient_id}: shape={data.shape}, min={data.min():.2f}, max={data.max():.2f}, non-zero={np.count_nonzero(data)}")

    except Exception as e:
        print(f" {patient_id}: Failed to load T1CE - {e}")
        problematic_patients.append(patient_id)

print("\nSummary:")
print(f"Total patients checked: {len(explorer.patient_dirs)}")
print(f"Patients with T1CE issues: {len(problematic_patients)}")
if problematic_patients:
    print("Problematic patients:", problematic_patients)


In [ ]:

DATA_ROOT = r"C:/Users/KamrunNaher/Downloads/BRaTS2021_Training_Data"
OUTPUT_ROOT = r"C:/Users/KamrunNaher/Downloads/BRaTS2021_processed"


if not os.path.exists(DATA_ROOT):
    print(f" ERROR: Data path not found: {DATA_ROOT}")
    print("Please update DATA_ROOT variable with correct path")
else:
    print(f" Data root found: {DATA_ROOT}")

print(f" Output will be saved to: {OUTPUT_ROOT}")

In [ ]:
class SimpleBRaTSExplorer:
    def __init__(self, data_root):
        """Simple BRaTS Data Explorer"""
        self.data_root = data_root
        self.modalities = ['t1', 't1ce', 't2', 'flair', 'seg']
        self.patient_dirs = []

    def discover_patients(self):
        """Find all patient directories"""
        print(" Discovering patient directories...")

        if not os.path.exists(self.data_root):
            print(f" Error: Directory {self.data_root} not found!")
            print(f"Current working directory: {os.getcwd()}")
            print(f"Contents of current directory:")
            for item in os.listdir('.'):
                print(f"   - {item}")
            return False


        all_items = os.listdir(self.data_root)
        self.patient_dirs = [item for item in all_items
                           if os.path.isdir(os.path.join(self.data_root, item))
                           and (item.startswith('BRaTS2021') or item.startswith('BraTS2021'))]
        self.patient_dirs.sort()

        print(f" Found {len(self.patient_dirs)} patient directories")

        if len(self.patient_dirs) > 0:
            print(f"   First patient: {self.patient_dirs[0]}")
            print(f"   Last patient: {self.patient_dirs[-1]}")

        return len(self.patient_dirs) > 0

    def check_sample_patients(self, sample_size=5):
        """Check structure of sample patients"""
        print(f"\n🔧 Checking sample patients...")

        check_patients = self.patient_dirs[:min(sample_size, len(self.patient_dirs))]

        for i, patient_id in enumerate(check_patients):
            patient_path = os.path.join(self.data_root, patient_id)
            print(f"\n   Patient {i+1}: {patient_id}")


            if os.path.exists(patient_path):
                files = os.listdir(patient_path)
                print(f"      Files found: {len(files)}")


                for modality in self.modalities:
                    expected_file = f"{patient_id}_{modality}.nii.gz"
                    if expected_file in files:
                        file_path = os.path.join(patient_path, expected_file)
                        file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
                        print(f"       {modality.upper():5}: {file_size:.1f} MB")
                    else:
                        print(f"       {modality.upper():5}: Missing!")
            else:
                print(f"       Patient directory not found!")

    def try_load_nibabel(self):
        """Try to load nibabel and analyze one image"""
        print(f"\n Attempting to load medical images...")

        if not NIBABEL_AVAILABLE:
            print(" nibabel not available - skipping image analysis")
            return

        try:

            if len(self.patient_dirs) > 0:
                patient_id = self.patient_dirs[0]
                patient_path = os.path.join(self.data_root, patient_id)


                t1_file = os.path.join(patient_path, f"{patient_id}_t1.nii.gz")

                if os.path.exists(t1_file):
                    print(f"   Loading: {patient_id}_t1.nii.gz")

                    img = nib.load(t1_file)
                    data = img.get_fdata()

                    print(f"    Image shape: {data.shape}")
                    print(f"    Data type: {data.dtype}")
                    print(f"    Value range: [{np.min(data):.1f}, {np.max(data):.1f}]")
                    print(f"    Non-zero voxels: {np.count_nonzero(data)}")


                    self.simple_visualization(data, patient_id)

                else:
                    print(f"    T1 file not found: {t1_file}")

        except Exception as e:
            print(f" Error loading image: {e}")

    def simple_visualization(self, data, patient_id):
        """Create simple visualization"""
        try:
            print(f"    Creating visualization...")


            mid_x, mid_y, mid_z = [s // 2 for s in data.shape]

            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            fig.suptitle(f'Patient {patient_id} - T1 MRI Slices', fontsize=14)


            axes[0].imshow(data[:, :, mid_z], cmap='gray')
            axes[0].set_title(f'Axial (z={mid_z})')
            axes[0].axis('off')


            axes[1].imshow(data[mid_x, :, :], cmap='gray')
            axes[1].set_title(f'Sagittal (x={mid_x})')
            axes[1].axis('off')


            axes[2].imshow(data[:, mid_y, :], cmap='gray')
            axes[2].set_title(f'Coronal (y={mid_y})')
            axes[2].axis('off')

            plt.tight_layout()
            plt.show()
            print("    Visualization created!")

        except Exception as e:
            print(f"    Visualization failed: {e}")

    def generate_summary(self):
        """Generate basic summary"""
        print(f"\n{'='*60}")
        print(f"BRATS 2021 BASIC EXPLORATION SUMMARY")
        print(f"{'='*60}")
        print(f" Data Root: {os.path.abspath(self.data_root)}")
        print(f" Total Patients Found: {len(self.patient_dirs)}")
        print(f" Expected Modalities: {', '.join([m.upper() for m in self.modalities])}")

        if len(self.patient_dirs) > 0:
            print(f" Patient ID Range: {self.patient_dirs[0]} → {self.patient_dirs[-1]}")


            if len(self.patient_dirs) > 0:
                sample_patient = os.path.join(self.data_root, self.patient_dirs[0])
                if os.path.exists(sample_patient):
                    sample_files = os.listdir(sample_patient)
                    sample_size = sum(os.path.getsize(os.path.join(sample_patient, f))
                                    for f in sample_files if f.endswith('.gz'))
                    total_size_gb = (sample_size * len(self.patient_dirs)) / (1024**3)
                    print(f" Estimated Dataset Size: {total_size_gb:.1f} GB")

        print(f"{'='*60}")

print(" Data Explorer class defined")

In [ ]:
print(" Starting BRaTS 2021 Data Exploration")
print("=" * 50)

explorer = SimpleBRaTSExplorer(DATA_ROOT)


if not explorer.discover_patients():
    print("\n Could not find patient data. Please check your path!")
    print(f"Current path being checked: {os.path.abspath(DATA_ROOT)}")
else:

    explorer.check_sample_patients()


    explorer.try_load_nibabel()


    explorer.generate_summary()

    print(f"\n Basic exploration completed!")

In [ ]:
class BRaTSPreprocessor:
    def __init__(self, data_root, output_root=None):

        self.data_root = Path(data_root)
        self.output_root = Path(output_root) if output_root else Path(data_root).parent / "BRaTS2021_Processed"
        self.modalities = ['t1', 't1ce', 't2', 'flair']
        self.target = 'seg'


        self.output_root.mkdir(parents=True, exist_ok=True)


        self.patient_dirs = [d for d in self.data_root.iterdir()
                           if d.is_dir() and (d.name.startswith('BRaTS2021') or d.name.startswith('BraTS2021'))]
        self.patient_dirs.sort()

        print(f" Preprocessor initialized")
        print(f"    Input: {self.data_root}")
        print(f"    Output: {self.output_root}")
        print(f"    Patients: {len(self.patient_dirs)}")

    def load_patient_data(self, patient_id):

        patient_path = self.data_root / patient_id

        data = {}
        affines = {}
        headers = {}


        for modality in self.modalities + [self.target]:
            file_path = patient_path / f"{patient_id}_{modality}.nii.gz"

            if file_path.exists():
                img = nib.load(str(file_path))
                data[modality] = img.get_fdata().astype(np.float32)
                affines[modality] = img.affine
                headers[modality] = img.header
            else:
                print(f" Missing file: {file_path}")
                return None, None, None

        return data, affines, headers

    def normalize_intensity(self, image, method='z_score', mask=None):
        np.array: Normalized image

        Create simple brain mask from T1 image

        Args:
            t1_image (np.array): T1-weighted image
            threshold (float): Threshold for brain extraction

        Returns:
            np.array: Binary brain mask

        Preprocess a single patient's data

        Args:
            patient_id (str): Patient identifier
            normalization (str): Normalization method

        Returns:
            dict: Processed data


In [ ]:
np.mean(processed_data['t1ce']), np.std(processed_data['t1ce'])


In [ ]:

print(" Initializing Preprocessor...")
preprocessor = BRaTSPreprocessor(DATA_ROOT, OUTPUT_ROOT)


if NIBABEL_AVAILABLE and len(preprocessor.patient_dirs) > 0:
    print("\n Preprocessing Visualization Demo")
    sample_patient = preprocessor.patient_dirs[0].name
    preprocessor.visualize_preprocessing(sample_patient)
else:
    print(" Skipping preprocessing demo - nibabel not available or no patients found")

In [ ]:

sample_patient = preprocessor.patient_dirs[0]
if isinstance(sample_patient, Path):
    sample_patient = sample_patient.name

preprocessor.visualize_preprocessing(sample_patient)


In [ ]:
for mod in ['t1', 't1ce', 't2', 'flair']:
    img = processed_data[mod]
    img = (img - np.mean(img)) / (np.std(img) + 1e-8)
    processed_data[mod] = img


In [ ]:
class BRaTSDataSplitter:
    def __init__(self, data_root, output_root=None):

        self.data_root = Path(data_root)
        self.output_root = Path(output_root) if output_root else Path(data_root).parent / "BRaTS2021_Splits"
        self.output_root.mkdir(parents=True, exist_ok=True)


        self.patient_dirs = [d for d in self.data_root.iterdir()
                           if d.is_dir() and (d.name.startswith('BRaTS2021') or d.name.startswith('BraTS2021'))]
        self.patient_dirs.sort()


        self.patient_metadata = pd.DataFrame()

        print(f" Data Splitter initialized")
        print(f"    Total patients: {len(self.patient_dirs)}")

    def extract_patient_metadata(self):

        print(f" Extracting patient metadata...")

        if not NIBABEL_AVAILABLE:
            print(" nibabel not available - using basic metadata only")

            metadata_list = []
            for patient_dir in self.patient_dirs:
                patient_id = patient_dir.name
                patient_num = int(patient_id.split('_')[-1])
                institution_group = patient_num // 100

                metadata = {
                    'patient_id': patient_id,
                    'patient_number': patient_num,
                    'institution_group': institution_group,
                    'has_tumor': True,
                    'tumor_volume': 1000,
                    'enhancing_volume': 100,
                    'core_volume': 500,
                    'whole_volume': 1000,
                    'tumor_complexity': 3,
                    'enhancing_ratio': 0.1,
                    'core_ratio': 0.5
                }
                metadata_list.append(metadata)

            self.patient_metadata = pd.DataFrame(metadata_list)
            return self.patient_metadata

        metadata_list = []

        for i, patient_dir in enumerate(self.patient_dirs):
            patient_id = patient_dir.name

            try:

                seg_path = patient_dir / f"{patient_id}_seg.nii.gz"
                if seg_path.exists():
                    seg_img = nib.load(str(seg_path))
                    seg_data = seg_img.get_fdata()


                    has_tumor = np.any(seg_data > 0)
                    tumor_volume = np.sum(seg_data > 0)
                    enhancing_volume = np.sum(seg_data == 4)
                    core_volume = np.sum((seg_data == 1) | (seg_data == 4))
                    whole_volume = np.sum(seg_data > 0)


                    unique_labels = len(np.unique(seg_data[seg_data > 0]))


                    patient_num = int(patient_id.split('_')[-1])
                    institution_group = patient_num // 100

                    metadata = {
                        'patient_id': patient_id,
                        'patient_number': patient_num,
                        'has_tumor': has_tumor,
                        'tumor_volume': tumor_volume,
                        'enhancing_volume': enhancing_volume,
                        'core_volume': core_volume,
                        'whole_volume': whole_volume,
                        'tumor_complexity': unique_labels,
                        'institution_group': institution_group,
                        'enhancing_ratio': enhancing_volume / max(whole_volume, 1),
                        'core_ratio': core_volume / max(whole_volume, 1)
                    }

                    metadata_list.append(metadata)

            except Exception as e:
                print(f"    Error processing {patient_id}: {e}")


            if (i + 1) % 100 == 0:
                print(f"   Processed {i + 1}/{len(self.patient_dirs)} patients...")

        self.patient_metadata = pd.DataFrame(metadata_list)

        print(f" Metadata extracted for {len(self.patient_metadata)} patients")
        return self.patient_metadata

    def create_stratified_splits(self, test_size=0.15, val_size=0.15, random_state=42):

        print(f"\n Creating Stratified Data Splits...")

        if self.patient_metadata.empty:
            self.extract_patient_metadata()


        df = self.patient_metadata.copy()


        volume_terciles = df['tumor_volume'].quantile([0.33, 0.66])
        df['volume_category'] = pd.cut(df['tumor_volume'],
                                     bins=[0, volume_terciles.iloc[0], volume_terciles.iloc[1], float('inf')],
                                     labels=['small', 'medium', 'large'])


        df['stratify_key'] = (df['institution_group'].astype(str) + '_' +
                             df['volume_category'].astype(str) + '_' +
                             df['tumor_complexity'].astype(str))


        stratum_counts = df['stratify_key'].value_counts()
        large_strata = stratum_counts[stratum_counts >= 3].index
        df_filtered = df[df['stratify_key'].isin(large_strata)]

        print(f"    Using {len(df_filtered)}/{len(df)} patients for stratified split")
        print(f"    Stratification groups: {len(large_strata)}")


        X = df_filtered[['patient_id']].values
        y = df_filtered['stratify_key'].values

        try:
            X_trainval, X_test, y_trainval, y_test = train_test_split(
                X, y, test_size=test_size,
                stratify=y, random_state=random_state
            )


            val_size_adjusted = val_size / (1 - test_size)
            X_train, X_val, y_train, y_val = train_test_split(
                X_trainval, y_trainval, test_size=val_size_adjusted,
                stratify=y_trainval, random_state=random_state
            )

        except ValueError as e:
            print(f"    Stratified split failed, using random split: {e}")


            X_trainval, X_test = train_test_split(
                X, test_size=test_size, random_state=random_state
            )
            X_train, X_val = train_test_split(
                X_trainval, test_size=val_size_adjusted, random_state=random_state
            )


        train_patients = [x[0] for x in X_train]
        val_patients = [x[0] for x in X_val]
        test_patients = [x[0] for x in X_test]


        splits = {
            'train': train_patients,
            'validation': val_patients,
            'test': test_patients
        }


        print(f"\n Data Split Summary:")
        print(f"    Training:   {len(train_patients):4d} patients ({len(train_patients)/len(df_filtered)*100:.1f}%)")
        print(f"    Validation: {len(val_patients):4d} patients ({len(val_patients)/len(df_filtered)*100:.1f}%)")
        print(f"    Testing:    {len(test_patients):4d} patients ({len(test_patients)/len(df_filtered)*100:.1f}%)")


        self.save_splits(splits)

        return splits

    def save_splits(self, splits):

        print(f"\n Saving splits to disk...")


        splits_json_path = self.output_root / "data_splits.json"
        with open(splits_json_path, 'w') as f:
            json.dump(splits, f, indent=2)


        for split_name, patient_list in splits.items():
            split_file = self.output_root / f"{split_name}_patients.txt"
            with open(split_file, 'w') as f:
                for patient_id in patient_list:
                    f.write(f"{patient_id}\n")


        if not self.patient_metadata.empty:
            metadata_path = self.output_root / "patient_metadata.csv"
            self.patient_metadata.to_csv(metadata_path, index=False)

        print(f"Files saved to: {self.output_root}")

    def create_data_loader_config(self):

        print(f"\n Creating Data Loader Configuration...")

        config = {
            'dataset_info': {
                'name': 'BRaTS 2021',
                'total_patients': len(self.patient_dirs),
                'modalities': ['t1', 't1ce', 't2', 'flair'],
                'target': 'seg',
                'image_shape': [240, 240, 155],
                'voxel_spacing': [1.0, 1.0, 1.0],
                'num_classes': 4
            },
            'paths': {
                'data_root': str(self.data_root),
                'splits_root': str(self.output_root),
                'processed_root': str(self.data_root.parent / "BRaTS2021_Processed")
            },
            'preprocessing': {
                'normalization': 'z_score',
                'brain_extraction': True,
                'intensity_clipping': [-5, 5],
                'target_size': [240, 240, 155]
            },
            'augmentation': {
                'rotation_range': 10,
                'zoom_range': [0.9, 1.1],
                'brightness_range': [0.8, 1.2],
                'flip_probability': 0.5
            },
            'training': {
                'batch_size': 2,
                'learning_rate': 0.001,
                'epochs': 100,
                'early_stopping_patience': 15,
                'loss_function': 'dice_ce',
                'metrics': ['dice', 'hausdorff', 'sensitivity', 'specificity']
            }
        }


        config_path = self.output_root / "training_config.json"
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)

        print(f"    Configuration saved: {config_path}")
        return config

print(" Data Splitter class defined")


In [ ]:

print(" Starting Data Splitting Pipeline")
print("=" * 50)


SPLITS_OUTPUT = os.path.join(os.path.dirname(DATA_ROOT), "BRaTS2021_Splits")

splitter = BRaTSDataSplitter(DATA_ROOT, SPLITS_OUTPUT)


print("\n Step 1: Extracting Patient Metadata")
metadata = splitter.extract_patient_metadata()


if not metadata.empty:
    print(f"\n Dataset Statistics:")
    print(f"   Total patients with metadata: {len(metadata)}")
    print(f"   Average tumor volume: {metadata['tumor_volume'].mean():.0f} voxels")
    print(f"   Institution groups: {sorted(metadata['institution_group'].unique())}")


    print("\n Step 2: Creating Train/Val/Test Splits")
    splits = splitter.create_stratified_splits(
        test_size=0.15,
        val_size=0.15,
        random_state=42
    )


    print("\n Step 3: Creating Training Configuration")
    config = splitter.create_data_loader_config()

    print(f"\n Data splitting completed successfully!")
    print(f" All files saved to: {SPLITS_OUTPUT}")

else:
    print(" Could not extract metadata - check nibabel installation or data paths")

In [ ]:
def plot_dataset_overview(metadata_df):

    if metadata_df.empty:
        print(" No metadata available for visualization")
        return

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('BRaTS 2021 Dataset Overview', fontsize=16)


    inst_counts = metadata_df['institution_group'].value_counts().sort_index()
    axes[0, 0].bar(inst_counts.index, inst_counts.values, color='skyblue')
    axes[0, 0].set_title('Patients per Institution Group')
    axes[0, 0].set_xlabel('Institution Group')
    axes[0, 0].set_ylabel('Number of Patients')


    axes[0, 1].hist(metadata_df['tumor_volume'], bins=30, alpha=0.7, color='lightgreen')
    axes[0, 1].set_title('Tumor Volume Distribution')
    axes[0, 1].set_xlabel('Tumor Volume (voxels)')
    axes[0, 1].set_ylabel('Frequency')


    axes[1, 0].hist(metadata_df['enhancing_ratio'], bins=30, alpha=0.7, color='orange')
    axes[1, 0].set_title('Enhancing Tumor Ratio Distribution')
    axes[1, 0].set_xlabel('Enhancing Ratio')
    axes[1, 0].set_ylabel('Frequency')

    complexity_counts = metadata_df['tumor_complexity'].value_counts().sort_index()
    axes[1, 1].bar(complexity_counts.index, complexity_counts.values, color='lightcoral')
    axes[1, 1].set_title('Tumor Complexity Distribution')
    axes[1, 1].set_xlabel('Number of Tumor Regions')
    axes[1, 1].set_ylabel('Number of Patients')

    plt.tight_layout()
    plt.show()

def analyze_split_balance(splits, metadata_df):
    """Analyze balance across train/val/test splits"""
    if metadata_df.empty:
        print(" No metadata available for split analysis")
        return

    print("\n Split Balance Analysis:")
    print("=" * 50)

    for split_name, patient_list in splits.items():
        split_meta = metadata_df[metadata_df['patient_id'].isin(patient_list)]

        if len(split_meta) > 0:
            print(f"\n{split_name.upper()} SET:")
            print(f"   Patients: {len(split_meta)}")
            print(f"   Avg tumor volume: {split_meta['tumor_volume'].mean():.0f}")
            print(f"   Avg enhancing ratio: {split_meta['enhancing_ratio'].mean():.3f}")
            print(f"   Institution distribution: {dict(split_meta['institution_group'].value_counts())}")

print(" Visualization functions defined")

In [ ]:

try:
    if 'splits' in locals() and 'metadata' in locals():
        print(" Creating Dataset Visualizations...")
        plot_dataset_overview(metadata)

        print("\n Analyzing Split Balance...")
        analyze_split_balance(splits, metadata)
    else:
        print(" Skipping analysis - run previous cells first to generate splits and metadata")
except Exception as e:
    print(f" Analysis failed: {e}")

In [ ]:
def load_existing_splits(splits_dir):
    """Load previously saved splits from disk"""
    splits_dir = Path(splits_dir)

    if not splits_dir.exists():
        print(f" Splits directory not found: {splits_dir}")
        return None


    splits_json = splits_dir / "data_splits.json"
    if splits_json.exists():
        with open(splits_json, 'r') as f:
            splits = json.load(f)
        print(f" Loaded splits from: {splits_json}")


        total_patients = sum(len(patient_list) for patient_list in splits.values())
        print(f" Split Summary:")
        for split_name, patient_list in splits.items():
            print(f"   {split_name}: {len(patient_list)} patients ({len(patient_list)/total_patients*100:.1f}%)")

        return splits
    else:
        print(f" Splits file not found: {splits_json}")
        return None

